In [ ]:
# ゲートのテストを定義しましょう
def test_gate(gate, pin1, pin2, expected):
    print("Testing %s(%s, %s) = %s ... " % (gate.__name__, pin1, pin2, expected), end='')
    if gate(pin1, pin2) == expected:
        print("Success")
    else:
        print("Failed")

In [ ]:
# XOR gate
def xor_gate(pin1, pin2):
    return (pin1 and not pin2) or (not pin1 and pin2)

In [ ]:
test_gate(xor_gate, True, True, False)
test_gate(xor_gate, True, False, True)
test_gate(xor_gate, False, False, False)
test_gate(xor_gate, False, True, True)

In [ ]:
import math
import random

# [pin1, pin2, expected]
dataset = [
    [1.0, 1.0, 0.0],
    [1.0, 0.0, 1.0],
    [0.0, 0.0, 0.0],
    [0.0, 1.0, 1.0],
]

true_threshold = 0.95
for generation in range(10000):
    pin1 = random.random() * (1 - true_threshold) + random.sample([true_threshold, 0.0], 1)[0]
    pin2 = random.random() * (1 - true_threshold) + random.sample([true_threshold, 0.0], 1)[0]
    if (pin1 > true_threshold and pin2 < (1 - true_threshold)) or (pin1 < (1 - true_threshold) and pin2 > true_threshold):
        expect = 1.0
    else:
        expect = 0.0
    #print("%.2f, %.2f -> %.2f" % (pin1, pin2, expect))
    dataset.append([pin1, pin2, expect])

split = math.floor(len(dataset) * 0.8)
training_data = dataset[0:split]
test_data = dataset[split:]

print("Training data: %d items." % len(training_data))
print("Testing data: %d items." % len(test_data))

# ANNゲートのテストを定義しましょう
def test_ann_gate(got, pin1, pin2, expected, tolerance):
    if got < expected + tolerance and got > expected - tolerance:
        return True
    else:
        return False

In [ ]:
import tensorflow as tf

In [ ]:
def train_and_evaluate(graph, tolerance=0.1):
    with tf.Session(graph=graph) as session:
        session.run(tf.global_variables_initializer())
        for idx, data in enumerate(training_data):
            _, current_loss = session.run([trainer, loss],
                                    feed_dict={
                                        X: data[0:2],
                                        y_expected: data[-1]
                                    })
        print("Final loss: %.10f" % current_loss)

        successes = 0
        for data in training_data:
            result = session.run(y, feed_dict={X: data[0:2]})
            if test_ann_gate(result[0], data[0], data[1], data[2], tolerance):
                successes += 1
        print("%.2f%% self successes." % (100.0 * successes / len(training_data)))
        
        successes = 0
        for data in test_data:
            result = session.run(y, feed_dict={X: data[0:2]})
            if test_ann_gate(result[0], data[0], data[1], data[2], tolerance):
                successes += 1
        print("%.2f%% test successes." % (100.0 * successes / len(test_data)))

In [ ]:
no_hidden = tf.Graph()
with no_hidden.as_default():
    Weights = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
    biases = tf.Variable(tf.zeros([1]))
    
    X = tf.placeholder(tf.float32, shape=(2))
    y_expected = tf.placeholder(tf.float32)
    y = tf.sigmoid(tf.tensordot(Weights, X, axes=1) + biases)
    
    loss = tf.reduce_mean(tf.square(y - y_expected))
    optimizer = tf.train.GradientDescentOptimizer(0.2)
    trainer = optimizer.minimize(loss)

for tolerance in [0.1, 0.01, 0.001]:
    print("Tolerance: %s" % tolerance)
    train_and_evaluate(no_hidden, tolerance)
    print()

In [ ]:
simple_hidden = tf.Graph()
with simple_hidden.as_default():
    front_layer = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0))
    front_biases = tf.Variable(tf.zeros([1]))
    hidden_layer = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
    hidden_biases = tf.Variable(tf.zeros([1]))
    
    X = tf.placeholder(tf.float32, shape=(2))
    y_expected = tf.placeholder(tf.float32)
    inner_y = tf.sigmoid(tf.tensordot(front_layer, X, axes=1) + front_biases)
    y = tf.sigmoid(tf.tensordot(hidden_layer, inner_y, axes=1) + hidden_biases)
    
    loss = tf.reduce_mean(tf.square(y - y_expected))
    optimizer = tf.train.GradientDescentOptimizer(0.2)
    trainer = optimizer.minimize(loss)

for tolerance in [0.1, 0.01, 0.001]:
    print("Tolerance: %s" % tolerance)
    train_and_evaluate(simple_hidden, tolerance)
    print()